## Install MMAction2

In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Jun_13_19:42:34_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.91
Build cuda_12.2.r12.2/compiler.32965470_0


'gcc' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
# Install mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2

J:\PERSONAL\CURSOS\CAPSTONE-FITIZENS\notebooks\mmaction2


Cloning into 'mmaction2'...
C:\Users\jorge\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
!pip install -e .

# Install some optional requirements
!pip install -r requirements/optional.txt

In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

2.2.0 True


In [1]:
# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

1.2.0


In [13]:
# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

not available
MSVC 192729111


In [ ]:
# Check MMEngine installation
from mmengine.utils.dl_utils import collect_env
print(collect_env())

## Perform inference with a MMAction2 recognizer
MMAction2 already provides high level APIs to do inference and training.

In [12]:
# !mkdir checkpoints
!wget -c https://download.openmmlab.com/mmaction/v1.0/recognition/uniformerv2/kinetics600/uniformerv2-large-p14-res224_clip-kinetics710-pre_u8_kinetics600-rgb_20221219-cf88e4c2.pth \
      -O checkpoints/uniformerv2-large-p14-res224_clip-kinetics710-pre_u8_kinetics600-rgb_20221219-cf88e4c2.pth

A subdirectory or file checkpoints already exists.


^C


In [ ]:
%pip install timm

In [5]:
from mmaction.apis import inference_recognizer, init_recognizer
from mmengine import Config

# Choose to use a config and initialize the recognizer
config = 'mmaction2/configs/recognition/uniformerv2/uniformerv2-large-p14-res224_clip-kinetics710-pre_u8_kinetics600-rgb.py'
config = Config.fromfile(config)
# Setup a checkpoint file to load
checkpoint = 'checkpoints/uniformerv2-large-p14-res224_clip-kinetics710-pre_u8_kinetics600-rgb_20221219-cf88e4c2.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0')

02/10 14:13:17 - mmengine - INFO - Drop path rate: 0.0
02/10 14:13:17 - mmengine - INFO - No L_MHRA: True
02/10 14:13:17 - mmengine - INFO - Double L_MHRA: True
02/10 14:13:17 - mmengine - INFO - Drop path rate: 0.0
02/10 14:13:17 - mmengine - INFO - No L_MHRA: True
02/10 14:13:17 - mmengine - INFO - Double L_MHRA: True
02/10 14:13:17 - mmengine - INFO - Drop path rate: 0.0
02/10 14:13:17 - mmengine - INFO - No L_MHRA: True
02/10 14:13:17 - mmengine - INFO - Double L_MHRA: True
02/10 14:13:18 - mmengine - INFO - Drop path rate: 0.0
02/10 14:13:18 - mmengine - INFO - No L_MHRA: True
02/10 14:13:18 - mmengine - INFO - Double L_MHRA: True
02/10 14:13:18 - mmengine - INFO - Drop path rate: 0.0
02/10 14:13:18 - mmengine - INFO - No L_MHRA: True
02/10 14:13:18 - mmengine - INFO - Double L_MHRA: True
02/10 14:13:18 - mmengine - INFO - Drop path rate: 0.0
02/10 14:13:18 - mmengine - INFO - No L_MHRA: True
02/10 14:13:18 - mmengine - INFO - Double L_MHRA: True
02/10 14:13:18 - mmengine - INFO -

In [10]:
# Use the recognizer to do inference
from operator import itemgetter
video = '../videos/PUSH_UP_SIT_UP.mp4'
label = 'mmaction2/tools/data/kinetics/label_map_k600.txt'
results = inference_recognizer(model, video)

pred_scores = results.pred_score.tolist()
score_tuples = tuple(zip(range(len(pred_scores)), pred_scores))
score_sorted = sorted(score_tuples, key=itemgetter(1), reverse=True)
top5_label = score_sorted[:5]

labels = open(label).readlines()
labels = [x.strip() for x in labels]
results = [(labels[k[0]], k[1]) for k in top5_label]


In [11]:
print('The top-5 labels with corresponding scores are:')
for result in results:
    print(f'{result[0]}: ', result[1])

The top-5 labels with corresponding scores are:
situp:  0.5068127512931824
push up:  0.44722211360931396
stretching arm:  0.006095241289585829
exercising with an exercise ball:  0.0042668539099395275
throwing ball (not baseball or American football):  0.0026855794712901115


In [12]:
results

[('situp', 0.5068127512931824),
 ('push up', 0.44722211360931396),
 ('stretching arm', 0.006095241289585829),
 ('exercising with an exercise ball', 0.0042668539099395275),
 ('throwing ball (not baseball or American football)', 0.0026855794712901115)]